<a href="https://colab.research.google.com/github/QasimKhan5x/Evaluation-Exercise/blob/main/Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
!pip install fastai --upgrade

Requirement already up-to-date: fastai in /usr/local/lib/python3.7/dist-packages (2.2.7)


In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

from fastai import learner
from fastai.data import core
from fastai.metrics import mse
from fastai.callback import schedule

In [59]:
# total columns = 49 (last is AW)
particles = pd.read_csv('particles.csv', usecols=list([0, 4, 5, 6, 7, 8, 9]), header=None, 
                        names=['eid', 'METphi', 'obj1', 'E1', 'pt1', 'eta1', 'phi1'])
particles

,eid,METphi,obj1,E1,pt1,eta1,phi1
0,354;monojet_Zp2000.0_DM_50.0;2.44336e-05;832841;-1.22431;b,1.64421;j,258722.0,243675.0,0.328962,2.25014;j,520092.0
1,354;monojet_Zp2000.0_DM_50.0;2.44336e-05;732115;1.17245;j,-1.84217;j,457647.0,110079.0,2.102870,2.17501;j,49649.1
2,354;monojet_Zp2000.0_DM_50.0;2.44336e-05;1.05628e+06;-3.06744;j,0.704554;j,1239300.0,602104.0,1.343240,-0.550856;j,193865.0
3,354;monojet_Zp2000.0_DM_50.0;2.44336e-05;631781;1.64181;j,-1.56592;j,75998.0,52304.3,0.905489,-0.806736;,NaN
4,354;monojet_Zp2000.0_DM_50.0;2.44336e-05;457316;-1.87536;j,1.23734;j,332790.0,126272.0,1.619980,-1.85524;,NaN
...,...,...,...,...,...,...,...
6394,354;monojet_Zp2000.0_DM_50.0;2.44336e-05;914355;1.52903;j,-1.77278;b,246762.0,95229.6,1.605110,-0.520258;j,402070.0
6395,354;monojet_Zp2000.0_DM_50.0;2.44336e-05;782256;-1.20751;j,1.83927;j,156224.0,152161.0,-0.222571,-1.83395;j,102296.0
6396,354;monojet_Zp2000.0_DM_50.0;2.44336e-05;645619;-2.48399;j,0.456786;j,408087.0,258903.0,-1.026900,1.53073;j,1428010.0
6397,354;monojet_Zp2000.0_DM_50.0;2.44336e-05;632390;-0.975687;j,2.56393;j,268828.0,259144.0,0.153813,0.578428;j,153340.0


In [60]:
particles['eid'] = particles['eid'].str.split(';').apply(lambda x : x[-3:]).apply(lambda x : ';'.join(x))
particles.head()

,eid,METphi,obj1,E1,pt1,eta1,phi1
0,832841;-1.22431;b,1.64421;j,258722.0,243675.0,0.328962,2.25014;j,520092.0
1,732115;1.17245;j,-1.84217;j,457647.0,110079.0,2.102870,2.17501;j,49649.1
2,1.05628e+06;-3.06744;j,0.704554;j,1239300.0,602104.0,1.343240,-0.550856;j,193865.0
3,631781;1.64181;j,-1.56592;j,75998.0,52304.3,0.905489,-0.806736;,NaN
4,457316;-1.87536;j,1.23734;j,332790.0,126272.0,1.619980,-1.85524;,NaN


In [61]:
particles['eidIsJ'] = particles['eid'].str.split(';').apply(lambda x : x[-1] == 'j')
particles['METphiIsJ'] = particles['METphi'].str.split(';').apply(lambda x : x[-1] == 'j')
particles['eta1IsJ'] = particles['eta1'].str.split(';').apply(lambda x : False if type(x) == type(1.0) else x[-1] == 'j')

particles.head()

,eid,METphi,obj1,E1,pt1,eta1,phi1,eidIsJ,METphiIsJ,eta1IsJ
0,832841;-1.22431;b,1.64421;j,258722.0,243675.0,0.328962,2.25014;j,520092.0,False,True,True
1,732115;1.17245;j,-1.84217;j,457647.0,110079.0,2.102870,2.17501;j,49649.1,True,True,True
2,1.05628e+06;-3.06744;j,0.704554;j,1239300.0,602104.0,1.343240,-0.550856;j,193865.0,True,True,True
3,631781;1.64181;j,-1.56592;j,75998.0,52304.3,0.905489,-0.806736;,NaN,True,True,False
4,457316;-1.87536;j,1.23734;j,332790.0,126272.0,1.619980,-1.85524;,NaN,True,True,False


In [62]:
particles.shape

(6399, 10)

In [63]:
particles[(particles['eidIsJ']) | (particles['METphiIsJ']) | (particles['eta1IsJ'])].shape

(6392, 10)

In [64]:
particles[(~particles['eidIsJ']) & (~particles['METphiIsJ']) & (~particles['eta1IsJ'])]

,eid,METphi,obj1,E1,pt1,eta1,phi1,eidIsJ,METphiIsJ,eta1IsJ
655,673825;0.390258;b,-2.1971;b,288278.0,276703.0,-0.106871,2.4419;b,116304.0,False,False,False
659,635439;-2.92769;b,0.206847;,NaN,NaN,NaN,NaN,NaN,False,False,False
1344,260068;-1.25616;b,1.8899;b,442099.0,436206.0,0.101171,-1.23124;,NaN,False,False,False
2074,638529;-2.6459;b,0.488063;,NaN,NaN,NaN,NaN,NaN,False,False,False
2779,681895;-0.349946;b,2.83166;,NaN,NaN,NaN,NaN,NaN,False,False,False
2872,953893;0.0928203;b,-3.04765;,NaN,NaN,NaN,NaN,NaN,False,False,False
5177,967728;0.852999;b,-2.28206;,NaN,NaN,NaN,NaN,NaN,False,False,False


In [65]:
particles1 = particles[(particles['eidIsJ']) | (particles['METphiIsJ']) | (particles['eta1IsJ'])]
particles1.reset_index(drop=True, inplace=True)
particles1.head()

,eid,METphi,obj1,E1,pt1,eta1,phi1,eidIsJ,METphiIsJ,eta1IsJ
0,832841;-1.22431;b,1.64421;j,258722.0,243675.0,0.328962,2.25014;j,520092.0,False,True,True
1,732115;1.17245;j,-1.84217;j,457647.0,110079.0,2.102870,2.17501;j,49649.1,True,True,True
2,1.05628e+06;-3.06744;j,0.704554;j,1239300.0,602104.0,1.343240,-0.550856;j,193865.0,True,True,True
3,631781;1.64181;j,-1.56592;j,75998.0,52304.3,0.905489,-0.806736;,NaN,True,True,False
4,457316;-1.87536;j,1.23734;j,332790.0,126272.0,1.619980,-1.85524;,NaN,True,True,False


In [66]:
particles1['eidIsJ'].value_counts()

True     6202
False     190
Name: eidIsJ, dtype: int64

In [68]:
particles1[~particles1['eidIsJ']]

,eid,METphi,obj1,E1,pt1,eta1,phi1,eidIsJ,METphiIsJ,eta1IsJ
0,832841;-1.22431;b,1.64421;j,258722.0,243675.0,0.328962,2.25014;j,520092.0,False,True,True
6,224967;1.49304;b,-2.69565;b,287305.0,246753.0,0.563157,-0.0624737;j,89818.0,False,False,True
29,757987;-0.953038;b,3.07611;j,606333.0,579035.0,0.273083,1.31517;j,51642.0,False,True,True
173,773428;0.356291;b,-2.28152;j,292305.0,243135.0,-0.610306,-2.86758;j,397080.0,False,True,True
233,595508;1.12034;b,-1.92827;j,464860.0,198326.0,-1.491750,-1.98288;j,50644.4,False,True,True
...,...,...,...,...,...,...,...,...,...,...
6266,578522;-2.81396;b,0.127856;j,353462.0,153338.0,-1.476420,2.694;j,698349.0,False,True,True
6326,397902;-1.52028;b,1.47404;j,315338.0,107474.0,1.736500,-2.66638;j,179767.0,False,True,True
6332,750931;-2.80122;b,0.215079;j,97964.8,91569.4,0.361601,-3.00871;j,191995.0,False,True,True
6334,382918;0.0088392;b,3.02817;j,212026.0,89959.0,-1.499820,-2.53556;j,80838.9,False,True,True


In [69]:
particles1['METphiIsJ'].value_counts()

True     5908
False     484
Name: METphiIsJ, dtype: int64

In [70]:
particles1['eta1IsJ'].value_counts()

True     4771
False    1621
Name: eta1IsJ, dtype: int64

In [71]:
particles1.shape

(6392, 10)

In [72]:
particles1['eid'].nunique()

6392